In [1]:
import pandas as pd
import zipfile
import requests
from io import BytesIO
import os

# Notice descriptive des datasets:
# https://www.data.gouv.fr/fr/datasets/r/d573456c-76eb-4276-b91c-e6b9c89d6656 (fichier pdf)

# URL of the CSV file inside a ZIP archive
url = "https://static.data.gouv.fr/resources/demandes-de-valeurs-foncieres/20241008-071041/valeursfoncieres-2023.txt.zip"
url = "https://www.data.gouv.fr/fr/datasets/r/dd516f7a-91bb-4cad-a63c-4b55cd457f4c"

# Directory and file paths
data_dir = "data"
csv_output_path = os.path.join(data_dir, "valeursfoncieres-2023.csv")
cleaned_csv_path = os.path.join(data_dir, "valeursfoncieres-2023-cleaned.csv")

# Create the data directory if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

# Download the ZIP file
response = requests.get(url)
response.raise_for_status()  # Raise an exception for HTTP errors

# Unzip the file
with zipfile.ZipFile(BytesIO(response.content)) as z:
    
    #csv_filename = z.namelist()[0]
    #with z.open(csv_filename) as csv_file:
    #        # Save the CSV file to the data folder
    #        with open(csv_output_path, "wb") as output_file:
    #            output_file.write(csv_file.read())
    
    # Assuming the ZIP contains one file, extract and read it
    csv_filename = z.namelist()[0]
    with z.open(csv_filename) as csv_file:
        df = pd.read_csv(csv_file, sep='|', low_memory=False)  # Assuming '|' is the delimiter
        df = df[df['Commune'] == 'Montpellier']
        df.dropna(axis=1, how='all', inplace=True)

# Show the first few rows of the DataFrame
df.head()


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,1,05/01/2023,Vente,"1070000,00",184.0,NaN,ALL,0124,DES HETRES,1630.0,...,NaN,NaN,1,3.0,Dépendance,0.0,0.0,NaN,NaN,NaN
1,1,05/01/2023,Vente,"1070000,00",159.0,NaN,ALL,0124,DES HETRES,1630.0,...,NaN,NaN,1,2.0,Appartement,233.0,8.0,NaN,NaN,NaN
2,1,05/01/2023,Vente,"1070000,00",159.0,NaN,ALL,0124,DES HETRES,1630.0,...,NaN,NaN,1,3.0,Dépendance,0.0,0.0,NaN,NaN,NaN
3,1,03/01/2023,Vente,"152200,00",2914.0,NaN,RTE,0107,DE PONCIN,1450.0,...,NaN,NaN,0,1.0,Maison,64.0,3.0,S,NaN,988.0
4,1,05/01/2023,Vente,"269000,00",427.0,T,CHE,0040,DE L'AUBEPIN,1800.0,...,NaN,NaN,0,1.0,Maison,73.0,3.0,S,NaN,835.0


In [2]:
print("Columns in the DataFrame:")
print(df.columns)

Columns in the DataFrame:
Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


In [11]:
# List the columns and the number of null values per column
null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100
null_summary = pd.DataFrame({
        "Null Count": null_counts,
        "Null Percentage": null_percentages.round(2)
}).sort_values(by="Null Count", ascending=False)

print("Columns with number of null values:")
print(null_summary)

Columns with number of null values:
                            Null Count  Null Percentage
Surface Carrez du 5eme lot     3798467            99.98
Surface Carrez du 4eme lot     3796531            99.92
No Volume                      3790969            99.78
5eme lot                       3790745            99.77
Surface Carrez du 3eme lot     3787066            99.68
4eme lot                       3779013            99.46
3eme lot                       3734247            98.28
Surface Carrez du 2eme lot     3689288            97.10
B/T/Q                          3631855            95.59
Nature culture speciale        3630026            95.54
Prefixe de section             3622138            95.33
Surface Carrez du 1er lot      3464107            91.17
2eme lot                       3433860            90.38
1er lot                        2623998            69.06
Surface reelle bati            1613870            42.48
Nombre pieces principales      1613870            42.48
Type local  

In [12]:
# Dans quel cas la valeur fonciere n' est pas renseignée ?
null_valeur_fonciere = df[df['Valeur fonciere'].isnull()]

# Display the rows with null 'valeur fonciere'
print("Rows where 'valeur fonciere' is null:")
print(null_valeur_fonciere)

Rows where 'valeur fonciere' is null:
         No disposition Date mutation Nature mutation Valeur fonciere  \
1004                  1    17/01/2023           Vente             NaN   
1005                  1    17/01/2023           Vente             NaN   
1006                  1    17/01/2023           Vente             NaN   
1012                  1    12/01/2023           Vente             NaN   
1013                  1    12/01/2023           Vente             NaN   
...                 ...           ...             ...             ...   
3791202               1    27/10/2023           Vente             NaN   
3791203               1    27/10/2023           Vente             NaN   
3791204               1    27/10/2023           Vente             NaN   
3799140               2    29/12/2023           Vente             NaN   
3799141               2    29/12/2023           Vente             NaN   

         No voie B/T/Q Type de voie Code voie         Voie  Code postal  ...  \
1004 

In [13]:
# Sauvegarder the cleaned dataset
try:
    df.to_csv(cleaned_csv_path, index=False, sep=';')
except Exception as e:
    print(f"An error occured: {e}")
